In [10]:
import pandas as pd
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle, json
from urllib import request, parse
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
from time import sleep

In [11]:
#pickle 데이터 불러오기
with open('Gameinfo_110000_to_120000.pickle', 'rb') as f:
    match_df = pickle.load(f)
#챔피언 id/이름 목록 파일 불러오기
with open('champion_id_and_name_list.pickle','rb') as f1:
    champ_id_dict=pickle.load(f1)

In [13]:
match_df.index = range(len(match_df))

In [14]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5424 entries, 0 to 5423
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   gameId                 5424 non-null   object
 1   gameCreation           5424 non-null   object
 2   gameDuration           5424 non-null   object
 3   queueId                5424 non-null   object
 4   gameVersion            5424 non-null   object
 5   teams                  5424 non-null   object
 6   participants           5424 non-null   object
 7   participantIdentities  5424 non-null   object
dtypes: object(8)
memory usage: 339.1+ KB


In [15]:
def parse_tup(summonerName, championId): #동원님 스크립트를 이용해서 함수 재조합.
    global driver, champ_id_dict
    name_parsed = parse.quote_plus(summonerName)
    champ = champ_id_dict[championId] #id -챔피언명 정보 딕셔너리로 불러와주세요. 딕셔너리 이름 다르게 했으면 이 이름으로 하셔야 합니다. 
    try:
        driver.get(f'https://www.op.gg/summoner/userName={name_parsed}')
        driver.find_element_by_xpath('//*[@id="right_gametype_soloranked"]').click()
        sleep(2)
        x=driver.find_element_by_xpath('//*[@id="GameAverageStatsBox-matches"]/div[1]/table/tbody/tr[2]/td[2]/div[2]')
        spans=x.find_elements_by_xpath('./span')
        recentKDA= float(spans[0].text.split(':')[0])
        recentW_R= int(spans[1].find_element_by_xpath('./span').text[:-1])
        totalW_R=int(driver.find_element_by_xpath('//*[@id="SummonerLayoutContent"]/div[2]/div[1]/div[1]/div/div[2]/div[3]/span[2]/span[3]').text.split()[-1][:-1])
        driver.get(f'https://www.op.gg/summoner/champions/userName={name_parsed}') #url 변경으로 대체가능
        driver.find_element_by_xpath('//*[@id="champion_season"]/li[2]/a').click()
        sleep(2)
        chamW_R=0
        chamKDA=0
        matchCNT=0
        table = driver.find_element_by_xpath('//*[@id="SummonerLayoutContent"]/div[3]/div/div/div[2]/div[1]/table/tbody')
        trs=table.find_elements_by_xpath('./tr')
        #픽 champion의 시즌 승률 검색 만약 데이터가 없으면 최근승률로 대체
        for tr in trs:
                tds = tr.find_elements_by_xpath('./td')
                if tds[2].get_attribute('data-value')==champ:
                    chamW_R=int(tds[3].find_element_by_xpath('.//span').text[:-1])
                    chamKDA=float(tds[4].get_attribute("data-value"))
                    divs=tds[3].find_elements_by_xpath('./div/div/div')
                    matchCNT=int(divs[1].text[:-1])+int(divs[3].text[:-1])
                    break
        else:
            chamW_R=recentW_R
            chamKDA=recentKDA
                
        
        return [summonerName,champ,chamW_R,chamKDA,recentW_R,recentKDA, totalW_R ,matchCNT]
    except: #추출 불가면 아무것도 return하지 않음.
        return 'err!'
    

In [16]:
data_list = []
data_list_failed = []

In [17]:
url_f = 'https://www.op.gg/summoner/userName={}'
#언어 설정 영어로 지정

i = 0
for i in range(0,len(match_df)-1,100):
    driver= webdriver.Chrome() #드라이버를 열어야 작동.
    driver.get('https://op.gg/')
    driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
    sleep(2)
    lang_list = driver.find_elements_by_class_name('setting-list__item')
    for lang in lang_list:
        if "English" in lang.text:
            lang.click()
    sleep(2)
    driver.find_elements_by_class_name('setting__button')[0].click()
    for j in tqdm(range(100)): # 100단위로 처리     
    #userlist parsing
        try: 
            team_blue = {'gameId':match_df.gameId[i+j], 'teamId':100 }
            team_red = {'gameId':match_df.gameId[i+j], 'teamId':200 }
            team_blue['win'] = True if match_df.teams[i+j][0]['win'] in ['Win', True, 'True'] else False
            team_red['win'] = True if match_df.teams[i+j][1]['win'] in ['Win', True, 'True'] else False
            #우선 둘이 같으면 데이터 오류로 판단하고 기각
            dat_err = False
            if team_blue['win'] == team_red['win']:
                dat_err = True
            team_blue['participants'] = []
            team_red['participants'] = []
            temp_blue =[]
            temp_red = []
            if not dat_err:
                for k in [0,5]: #blue : k=0, red : k=5
                    for l in range(5): #participantIdentities 에서 userid 가져옴,
                        p_info = match_df.participantIdentities[i+j][l+k]
                        p_name = p_info['player']['summonerName']
                        name_parsed = parse.quote_plus(p_name)
                        url = url_f.format(name_parsed)
                        html = request.urlopen(url).read().decode('utf8')
                        soup = BeautifulSoup(html, 'html.parser')
                        title = soup.title.string
                        if title[-1] == 'G': #반복 횟수 줄이기 위한 테크닉
                            break
                        elif k ==0: 
                            p_champ = match_df.participants[i+j][l]['championId']
                            tup = (p_name, p_champ)
                            temp_blue.append(tup)
                        elif k ==5: #페이지 유효
                            p_champ = match_df.participants[i+j][l+5]['championId']
                            tup = (p_name, p_champ)
                            temp_red.append(tup)


                if len(temp_blue)==5: #5명 모두 긁어올 때만
                    parsed_res =[]
                    for tup in temp_blue:
                        res = parse_tup(*tup)
                        parsed_res.append(res)
                        print(res)
                    if not ('err!' in parsed_res): 
                        team_blue['participants'] = parsed_res
                        data_list.append(team_blue)
                        #print('Blue add')

                if len(temp_red)==5: #5명 모두 긁어올 때만.
                    parsed_res =[]
                    for tup in temp_red:
                        res = parse_tup(*tup)
                        parsed_res.append(res)
                        print(res)
                    if not ('err!' in parsed_res): 
                        team_red['participants'] = parsed_res
                        data_list.append(team_red)
        except:
            data_list_failed.append(i+j)
        
    with open(f'Gameinfo_with_user_60000_to_70000_{i//100+1}.pickle', 'wb') as pk1: #100개 긁어올 때마다...
        pickle.dump(data_list, pk1)
        data_list = []
    
    driver.close() #100개 모을 때마다 강제휴식 
    sleep(10)
                                                  
        
            


 40%|█████████████████████████████████▌                                                  | 2/5 [00:12<00:20,  6.88s/it]

5
['vvanty0u', 'Kayn', 80, 3.26, 56, 3.03, 73, 5]
28
['스섹속', 'Taric', 61, 3.58, 40, 1.05, 50, 28]
err!
['잘까요', 'Lulu', 39, 2.14, 39, 2.14, 52, 0]


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:49<00:31, 15.93s/it]

41
['오연서남편', 'Draven', 59, 1.99, 52, 3.29, 56, 41]
275
['녹서스장군님', 'Katarina', 63, 2.55, 41, 2.42, 62, 275]
34
['졸려죽겠다진짜', 'Aatrox', 62, 2.27, 39, 2.11, 52, 34]
err!
err!


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:29<00:23, 23.22s/it]

24
['vvanty0u', 'Evelynn', 71, 4.13, 56, 3.03, 73, 24]
12
['자가최면', 'Lucian', 50, 2.51, 56, 2.67, 57, 12]
147
['고교 여신 김정근', 'Pyke', 49, 2.5, 53, 3.11, 51, 147]
17
['vvanty0u', 'Elise', 76, 4.45, 56, 3.03, 73, 17]
716
['진호우석옥천', 'Jax', 56, 1.57, 40, 1.58, 50, 716]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:12<00:00, 26.45s/it]

154
['Rh4psody', 'Akali', 53, 2.33, 48, 2.12, 49, 154]


In [ ]:
data_list

In [ ]:
with open('Gameinfo_with_user_60000_to_70000.pickle', 'wb') as pk1:
    pickle.dump(data_list, pk1)

In [ ]:
driver= webdriver.Chrome() #드라이버를 열어야 작동.
driver.get('https://op.gg/')
driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
sleep(2)
lang_list = driver.find_elements_by_class_name('setting-list__item')
for lang in lang_list:
    if "English" in lang.text:
        lang.click()
sleep(2)
driver.find_elements_by_class_name('setting__button')[0].click()
